<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/oilgasprocess1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install neqsim
"#!pip install -U pydantic

Save the fillowing fiel to a fluid1.e100 file

METRIC
-- Number of components:
NCOMPS
22 /
-- Equation of state
EOS
PR /
PRCORR
-- Reservoir temperature (C)
RTEMP
    112.30 /
-- Standard Conditions (C and bara)
STCOND
   15.00000    1.01325  /
-- Component names
CNAMES
N2
CO2
C1
C2
C3
iC4
C4
iC5
C5
C6
C7
C8
C9
C10-C12
C13-C14
C15-C16
C17-C19
C20-C22
C23-C25
C26-C28
C29-C33
C34-C80 /
-- Tc (K)
TCRIT
   126.200
   304.200
   190.600
   305.400
   369.800
   408.100
   425.200
   460.400
   469.600
   507.400
   527.669
   553.949
   579.636
   622.942
   666.834
   706.246
   747.081
   786.847
   821.763
   855.619
   894.027
  1061.556 /
-- Pc (Bar)
PCRIT
   33.9439
   73.7646
   46.0015
   48.8387
   42.4552
   36.4770
   37.9969
   33.8426
   33.7412
   29.6882
   30.7250
   27.8786
   25.1398
   21.6669
   19.1244
   17.4631
   16.1506
   15.1782
   14.5255
   14.0013
   13.5329
   12.5974 /
-- Omega
ACF
   0.04000
   0.22500
   0.00800
   0.09800
   0.15200
   0.17600
   0.19300
   0.22700
   0.25100
   0.29600
   0.32171
   0.36663
   0.41648
   0.50584
   0.60076
   0.69007
   0.78285
   0.87002
   0.94131
   1.00419
   1.06491
   1.04501 /
-- OmegaA
OMEGAA
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724
     0.45724 /
-- OmegaB
OMEGAB
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780
     0.07780 /
-- Molecular weights
MW
   28.0135
   44.0098
   16.0429
   30.0698
   44.0968
   58.1237
   58.1237
   72.1506
   72.1506
   85.1000
   91.3000
  104.7000
  119.8000
  147.7631
  180.7557
  213.2724
  249.8889
  288.2513
  323.6568
  359.5848
  401.4829
  565.6066 /
-- Boiling points (K)
TBOIL
    77.400
   194.650
   111.600
   184.600
   231.100
   261.400
   272.700
   301.000
   309.200
   341.900
   365.100
   389.900
   415.400
   462.297
   508.144
   547.043
   585.667
   623.616
   658.686
   691.204
   725.969
   817.977 /
-- Critical volumes (m3/kg-mole)
VCRIT
   0.08980
   0.09400
   0.09900
   0.14800
   0.20300
   0.26300
   0.25500
   0.30600
   0.30400
   0.37000
   0.45436
   0.48019
   0.53302
   0.63683
   0.75711
   0.88963
   1.05005
   1.22386
   1.38886
   1.56097
   1.76832
   2.86516 /
-- Critical Z-factors
ZCRIT
   0.29049
   0.27414
   0.28737
   0.28465
   0.28029
   0.28272
   0.27406
   0.27052
   0.26270
   0.26037
   0.31819
   0.29064
   0.27804
   0.26639
   0.26114
   0.26456
   0.27301
   0.28393
   0.29525
   0.30721
   0.32192
   0.40892 /
-- Volume translation/co-volume
SSHIFT
 -0.175888
 -0.045511
 -0.194020
 -0.143142
 -0.112702
 -0.099214
 -0.089659
 -0.070455
 -0.056872
  0.012573
  0.073720
  0.085173
  0.083037
  0.082231
  0.077598
  0.064618
  0.043364
  0.018779
 -0.003316
 -0.027041
 -0.053010
 -0.100845 /
-- Parachors (dyn/cm)
PARACHOR
    41.000
    78.000
    77.300
   108.900
   151.900
   181.500
   191.700
   225.000
   233.900
   271.000
   272.942
   304.298
   339.632
   407.069
   482.957
   559.054
   645.090
   734.682
   817.404
   901.299
  1000.122
  1473.547 /
-- Overall composition
ZI
  0.003570
  0.010280
  0.571653
  0.045892
  0.027131
  0.005420
  0.013881
  0.005730
  0.008050
  0.011760
  0.022521
  0.025721
  0.018431
  0.045402
  0.029981
  0.025641
  0.033021
  0.026731
  0.021591
  0.016941
  0.018971
  0.011680 /
-- Binary interaction coefficients for PR
BIC
 -0.0170
  0.0311  0.1200
  0.0515  0.1200  0.0000
  0.0852  0.1200  0.0000  0.0000
  0.1033  0.1200  0.0000  0.0000  0.0000
  0.0800  0.1200  0.0000  0.0000  0.0000  0.0000
  0.0922  0.1200  0.0000  0.0000  0.0000  0.0000  0.0000
  0.1000  0.1200  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1200  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000
/
BICS
 -0.0170
  0.0311  0.1200
  0.0515  0.1200  0.0000
  0.0852  0.1200  0.0000  0.0000
  0.1033  0.1200  0.0000  0.0000  0.0000
  0.0800  0.1200  0.0000  0.0000  0.0000  0.0000
  0.0922  0.1200  0.0000  0.0000  0.0000  0.0000  0.0000
  0.1000  0.1200  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1200  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0800  0.1000  0.0447  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000
  0.0000
/
-- Viscosity correlation
PEDERSEN
-- Volume translation/co-volume at surface conditions
SSHIFTS
 -0.175888
 -0.061524
 -0.194020
 -0.143142
 -0.112702
 -0.099214
 -0.089659
 -0.070455
 -0.056872
  0.012573
  0.056615
  0.079972
  0.090897
  0.104318
  0.108036
  0.101240
  0.085802
  0.065887
  0.046875
  0.026036
  0.002593
 -0.045217 /

In [4]:
"""
Oil process module
...
The module is used for establishing an Oil process,
setting input parameters and reading output from a simulation
"""
from functools import cache
from neqsim import jNeqSim
from pydantic.dataclasses import dataclass
from pydantic import Field
from annotated_types import Gt, Len
from neqsim.process import compressor, cooler, separator3phase, getProcess, clearProcess, mixer, heater, stream, pump, separator, runProcess, stream, valve, recycle2
from neqsim.thermo import fluid
from neqsim.thermo.thermoTools import readEclipseFluid
from pydantic import validator
@dataclass
class ProcessInput():
    """
    A class to define input parameters for the oil process.

    ...

    Attributes
    ----------
    feedGasFlowRateHP : float
        HP wells flow rate in unit MSm3/day
    feedGasFlowRateLP : float
        LP wells flow rate in unit MSm3/day
    """

    moleRateHP: list[float] = Field([0.01]*22, title="molar rates for22 components to HP manifold [mol/sec]")
    moleRateLP: list[float] = Field([0.01]*22, title="molar rates for22 components to LP manifold [mol/sec]")
    firstStagePressure: float = Field(
        60.0, ge=0.0, le=100.0, title="Pressure of first stage separator [bara]")
    firstStageTemperature: float = Field(
        70.0, ge=0.0, le=100.0, title="Temperature of first stage separator [C]")
    secondStagePressure: float = Field(
        30.0, ge=0.0, le=100.0, title="Second stage separator pressure [bara]")
    secondStageTemperature: float = Field(
        80.0, ge=0.0, le=100.0, title="Second stage separator temperature [C]")
    thirdStagePressure: float = Field(
        1.8, ge=0.0, le=100.0, title="Third stage separator pressure [bara]")
    thirdStageTemperature: float = Field(
        64.0, ge=0.0, le=100.0, title="Third stage separator temperature [C]")
    export_oil_temperature: float = Field(
        55.0, ge=0.0, le=100.0, title="export oil temperature [C]")
    export_oil_pressure: float = Field(
        8.8, ge=0.0, le=100.0, title="export oil pressure [bara]")
    firstStageRecompressorPressure: float = Field(
        6.5, ge=0.0, le=100.0, title="First stage recompression pressure [bara]")
    dew_point_scrubber_temperature: float = Field(
        30.0, ge=0.0, le=100.0, title="Dew point scrubber temperature [C]")
    export_gas_pressure: float = Field(
        130.0, ge=0.0, le=200.0, title="Export gas pressure [bara]")
    export_gas_temperature: float = Field(
        55.0, ge=0.0, le=100.0, title="Export gas temperature [C]")
    suctionCoolerTemperature: float = Field(
        30.0, ge=0.0, le=100.0, title="Compressor after cooler temperatures [C]")

    @validator('moleRateHP', 'moleRateLP')
    def check_mole_rates(cls, v):
        if len(v) != 22:
            raise ValueError('moleRateHP and moleRateLP must have 22 components')
        return v

@dataclass
class ProcessOutput():
    """
    A class to define output results from a Oil process simulation.

    ...

    Attributes
    ----------
    mass_balance _ float
        check of mass balance for simulation
    recompressor1_power : float
        power of first stage compressor (kW)
    recompressor2_power : float
        power of second stage compressor (kW)
    recompressor3_power : float
        power of third stage compressor (kW)
    exportcompressor_power : float
        power of export compressor (kW)
    gasexportflow : float
        gasexportflow (MSm3/day)
    oilexportflow : float
        oilexportflow (m3/hr)
    """
    mass_balance: float | None = None
    recompressor1_power: float | None = None
    recompressor2_power: float | None = None
    recompressor3_power: float | None = None
    exportcompressor_power: float | None = None
    gasexportflow: float | None = None
    oilexportflow: float | None = None




@cache
def getprocess():
    """
    The method creates a oil process object using neqsim
    """
    clearProcess()

    wellFluid = readEclipseFluid('fluid1.e300')

    LPwellFLuid = wellFluid.clone()

    wellStreamHP = stream(wellFluid)
    wellStreamHP.setName("HP well stream")
    try:
      wellStreamHP.getFluid().setMolarFlowRates(ProcessInput.moleRateHP)
    except:
      print('composition need to be set')
    wellStreamHP.setTemperature(ProcessInput.firstStageTemperature, "C")
    wellStreamHP.setPressure(ProcessInput.firstStagePressure, "bara")

    LPwellStream = stream(LPwellFLuid)
    LPwellStream.setName("LP well stream")
    try:
      LPwellStream.getFluid().setMolarFlowRates(ProcessInput.moleRateLP)
    except:
      print('composition need to be set')
    LPwellStream.setTemperature(ProcessInput.secondStageTemperature, "C")
    LPwellStream.setPressure(ProcessInput.secondStagePressure, "bara")

    firstStageSeparator = separator3phase(wellStreamHP)
    firstStageSeparator.setName("1st stage separator")

    oilHeaterFromFirstStage = heater(firstStageSeparator.getOilOutStream())
    oilHeaterFromFirstStage.setName("oil heater second stage")
    oilHeaterFromFirstStage.setOutTemperature(ProcessInput.secondStageTemperature,'C')
    oilHeaterFromFirstStage.setOutPressure(ProcessInput.secondStagePressure,'bara')

    secondStageSeparator = separator3phase(oilHeaterFromFirstStage.getOutStream())
    secondStageSeparator.addStream(LPwellStream)
    secondStageSeparator.setName("2nd stage separator")

    oilHeaterFromSecondStage = heater(secondStageSeparator.getOilOutStream())
    oilHeaterFromSecondStage.setName("oil heater third stage")
    oilHeaterFromSecondStage.setOutTemperature(ProcessInput.thirdStageTemperature,'C')
    oilHeaterFromSecondStage.setOutPressure(ProcessInput.thirdStagePressure,'bara')

    thirdStageSeparator = separator3phase(oilHeaterFromSecondStage.getOutStream())
    thirdStageSeparator.setName("3rd stage separator")

    oilThirdStageToSep =  wellStreamHP.clone()
    oilThirdStageToSep.setName("resyc oil")
    oilThirdStageToSep.setFlowRate(10.0, 'kg/hr')
    thirdStageSeparator.addStream(oilThirdStageToSep)

    exportoil = heater(thirdStageSeparator.getOilOutStream())
    exportoil.setName("export oil cooler")
    exportoil.setOutTemperature(ProcessInput.export_oil_temperature,'C')
    exportoil.setOutPressure(ProcessInput.export_oil_pressure,'bara')

    exportoilstream = stream(exportoil.getOutStream())
    exportoilstream.setName('export oil')

    firstStageCooler = cooler(thirdStageSeparator.getGasOutStream())
    firstStageCooler.setName("1st stage cooler")
    firstStageCooler.setOutTemperature(ProcessInput.suctionCoolerTemperature,'C')

    firstStageScrubber = separator(firstStageCooler.getOutStream())
    firstStageScrubber.setName("1st stage scrubber")

    firstStageCompressor = compressor(firstStageScrubber.getGasOutStream())
    firstStageCompressor.setName("1st stage compressor")
    firstStageCompressor.setOutletPressure(ProcessInput.firstStageRecompressorPressure)
    firstStageCompressor.setIsentropicEfficiency(0.75)

    firstStageCooler2 = cooler(firstStageCompressor.getOutStream())
    firstStageCooler2.setName("1st stage cooler2")
    firstStageCooler2.setOutTemperature(ProcessInput.suctionCoolerTemperature,'C')

    firstStageScrubber2 = separator(firstStageCooler2.getOutStream())
    firstStageScrubber2.setName("1st stage scrubber2")

    firstStageCompressor2 = compressor(firstStageScrubber2.getGasOutStream())
    firstStageCompressor2.setName("2nd stage compressor")
    firstStageCompressor2.setOutletPressure(ProcessInput.secondStagePressure)
    firstStageCompressor2.setIsentropicEfficiency(0.75)

    secondstagegasmixer = mixer("second Stage mixer")
    secondstagegasmixer.addStream(firstStageCompressor2.getOutStream())
    secondstagegasmixer.addStream(secondStageSeparator.getGasOutStream())

    secondStageCooler = cooler(secondstagegasmixer.getOutStream())
    secondStageCooler.setName("2nd stage cooler")
    secondStageCooler.setOutTemperature(ProcessInput.suctionCoolerTemperature,'C')

    secondStageScrubber = separator(secondStageCooler.getOutStream())
    secondStageScrubber.setName("2nd stage scrubber")

    secondStageCompressor = compressor(secondStageScrubber.getGasOutStream())
    secondStageCompressor.setName("3rd stage compressor")
    secondStageCompressor.setOutletPressure(ProcessInput.firstStagePressure)
    secondStageCompressor.setIsentropicEfficiency(0.75)

    richGasMixer = mixer("fourth Stage mixer")
    richGasMixer.addStream(secondStageCompressor.getOutStream())
    richGasMixer.addStream(firstStageSeparator.getGasOutStream())

    dewPointControlCooler = cooler(richGasMixer.getOutStream())
    dewPointControlCooler.setName("dew point cooler")
    dewPointControlCooler.setOutTemperature(ProcessInput.dew_point_scrubber_temperature,'C')

    dewPointScrubber = separator(dewPointControlCooler.getOutStream())
    dewPointScrubber.setName("dew point scrubber")

    lpLiqmixer = mixer("LP liq gas mixer");
    lpLiqmixer.addStream(firstStageScrubber.getLiquidOutStream());
    lpLiqmixer.addStream(firstStageScrubber2.getLiquidOutStream());
    lpLiqmixer.addStream(secondStageScrubber.getLiquidOutStream());
    lpLiqmixer.addStream(dewPointScrubber.getLiquidOutStream());

    lpResycle = recycle2("LP liq resycle")
    lpResycle.addStream(lpLiqmixer.getOutStream())
    lpResycle.setOutletStream(oilThirdStageToSep)
    lpResycle.setTolerance(1e-6)

    exportCompressor1 = compressor(dewPointScrubber.getGasOutStream())
    exportCompressor1.setName("export gas compressor")
    exportCompressor1.setOutletPressure(ProcessInput.export_gas_pressure, 'bara')
    exportCompressor1.setIsentropicEfficiency(0.75)

    exportGasCooler = cooler(exportCompressor1.getOutStream())
    exportGasCooler.setName("export gas cooler")
    exportGasCooler.setOutTemperature(ProcessInput.export_gas_temperature,'C')

    exportGas = stream(exportGasCooler.getOutStream())
    exportGas.setName("export gas")

    oilprocess = getProcess()
    return oilprocess


def updateinput(process, locinput):
    """
    update process with input parameters
    """
    process.getUnit('HP well stream').getFluid().setMolarFlowRates(locinput.moleRateHP)
    process.getUnit('HP well stream').setTemperature(locinput.firstStageTemperature, "C")
    process.getUnit('HP well stream').setPressure(locinput.firstStagePressure, "bara")
    process.getUnit('LP well stream').getFluid().setMolarFlowRates(locinput.moleRateLP)
    process.getUnit('LP well stream').setTemperature(ProcessInput.secondStageTemperature, "C")
    process.getUnit('LP well stream').setPressure(ProcessInput.secondStagePressure, "bara")
    process.getUnit('oil heater second stage').setOutTemperature(ProcessInput.secondStageTemperature,'C')
    process.getUnit('oil heater second stage').setOutPressure(ProcessInput.secondStagePressure,'bara')
    process.getUnit('oil heater third stage').setOutTemperature(ProcessInput.thirdStageTemperature,'C')
    process.getUnit('oil heater third stage').setOutPressure(ProcessInput.thirdStagePressure,'bara')
    process.getUnit('export oil cooler').setOutTemperature(ProcessInput.export_oil_temperature,'C')
    process.getUnit('export oil cooler').setOutPressure(ProcessInput.export_oil_pressure,'bara')
    process.getUnit('1st stage cooler').setOutTemperature(ProcessInput.suctionCoolerTemperature,'C')
    process.getUnit('1st stage compressor').setOutletPressure(ProcessInput.firstStageRecompressorPressure)
    process.getUnit('1st stage cooler2')
    process.getUnit('1st stage cooler2').setOutTemperature(ProcessInput.suctionCoolerTemperature,'C')
    process.getUnit('2nd stage compressor').setOutletPressure(ProcessInput.secondStagePressure)
    process.getUnit('2nd stage cooler').setOutTemperature(ProcessInput.suctionCoolerTemperature,'C')
    process.getUnit('3rd stage compressor').setOutletPressure(ProcessInput.firstStagePressure)
    process.getUnit('dew point cooler').setOutTemperature(ProcessInput.dew_point_scrubber_temperature,'C')
    process.getUnit('export gas compressor').setOutletPressure(ProcessInput.export_gas_pressure, 'bara')
    process.getUnit('export gas cooler').setOutTemperature(ProcessInput.export_gas_temperature,'C')



def getoutput():
    # update output
    #oilprocess.getUnit('HP well stream').getFluid().prettyPrint()
    outputparam = {
        'mass_balance': oilprocess.getUnit('HP well stream').getFlowRate('kg/hr')+oilprocess.getUnit('LP well stream').getFlowRate('kg/hr')-oilprocess.getUnit('export gas').getFlowRate('kg/hr')-oilprocess.getUnit('export oil').getFlowRate('kg/hr'),
        'recompressor1_power': oilprocess.getUnit('1st stage compressor').getPower()/1e3,
        'recompressor2_power': oilprocess.getUnit('2nd stage compressor').getPower()/1e3,
        'recompressor3_power': oilprocess.getUnit('3rd stage compressor').getPower()/1e3,
        'exportcompressor_power': oilprocess.getUnit('export gas compressor').getPower()/1e3,
        'oilexportflow':oilprocess.getUnit('export oil').getFlowRate('idSm3/hr')*24.0,
        'gasexportflow':oilprocess.getUnit('export gas').getFlowRate('MSm3/day'),
    }
    return outputparam


<ipython-input-4-6880d8f352e2>:60: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator('moleRateHP', 'moleRateLP')


In [5]:
# Create oil process (only do first time)
oilprocess = getprocess()

In [6]:
import numpy as np
HP1 = [22.38425926,	95.16550926,	5106.944444,	396.2615741,
       189.8888889,	24.08564815,	54.19907407,	15.47696759,
       19.22453704,	18.45486111,	22.85532407,	21.15277778,
       10.72013889,	20.27893519,	7.601041667,	2.752256944,
       1.550211806,	1.369644676,	0.94399294,	0.698867245,
       0.557957292,	0.714986574]

LP1 = [0.000001]*22

inputparam = {
    'moleRateHP': HP1,
    'moleRateLP': LP1,
    'firstStagePressure': 60.0,
    'firstStageTemperature': 70.0,
    'secondStagePressure': 30.0,
    'secondStageTemperature': 80.0,
    'thirdStagePressure': 1.8,
    'thirdStageTemperature': 64.0,
    'export_oil_temperature': 55.0,
    'export_oil_pressure': 8.8,
    'firstStageRecompressorPressure': 6.5,
    'dew_point_scrubber_temperature': 30.0,
    'export_gas_pressure': 130.0,
    'export_gas_temperature': 55.0,
    'suctionCoolerTemperature': 30.0
}

#update input to process calculations
updateinput(process=oilprocess, locinput=ProcessInput(**inputparam))

# run calculation for aximum 60 sec
thread = oilprocess.runAsThread()
thread.join(60*1000)
results = ProcessOutput(**getoutput())
print(results.__dict__)

{'mass_balance': 0.004227403587719891, 'recompressor1_power': 95.24749392259551, 'recompressor2_power': 96.20997393196859, 'recompressor3_power': 56.253115792605385, 'exportcompressor_power': 14074.49479865995, 'gasexportflow': 12.186035850585192, 'oilexportflow': 950.9445581823127}
